In [1]:
'''
Process-based analysis
Arrival process:
    The arrival process is a collection of arrival events
    This process starts when the environment starts running and will be running nonstop
Service Process:
    There are several events for this process
        Request the server
        Get the server
        Engage with the server
        Release with sever
'''

'\nProcess-based analysis\nArrival process:\n    The arrival process is a collection of arrival events\n    This process starts when the environment starts running and will be running nonstop\nService Process:\n    There are several events for this process\n        Request the server\n        Get the server\n        Engage with the server\n        Release with sever\n'

In [2]:
import numpy as np
import simpy

from scipy.stats import uniform,expon

### Example

simply ambulance get call and pick up patient

In [3]:
def arrival(env,ambulance):
    i = 0
    while True:
        dt = expon.rvs(scale = 1/1)
        yield env.timeout(dt)
        print(f"customer {i} has arrived at {env.now}")
        env.process(service(env,ambulance,i))
        i += 1
        
def service(env,ambulance,i):
    rqt = ambulance.request()
    print(f"customer {i} has {len(ambulance.queue)} before him")
    yield rqt
    print(f"customer {i} got to the ambulance")
    X = uniform.rvs()
    Y = uniform.rvs()
    dt = 2 * np.sqrt((X-0.5)**2 + (Y-0.5)**2)
    yield env.timeout(dt)
    print(f"customer {i} ends up in the hospotal")
    ambulance.release(rqt)


def system():
    env = simpy.Environment()
    ambulance = simpy.Resource(env,capacity = 1)    
    env.process(arrival(env,ambulance))
    env.run(until = 24)
 

In [4]:
system()

customer 0 has arrived at 0.5079296704013817
customer 0 has 0 before him
customer 0 got to the ambulance
customer 1 has arrived at 0.549417088836322
customer 1 has 1 before him
customer 2 has arrived at 0.7420435322697135
customer 2 has 2 before him
customer 3 has arrived at 1.1069859062261382
customer 3 has 3 before him
customer 0 ends up in the hospotal
customer 1 got to the ambulance
customer 1 ends up in the hospotal
customer 2 got to the ambulance
customer 2 ends up in the hospotal
customer 3 got to the ambulance
customer 4 has arrived at 2.337091673246784
customer 4 has 1 before him
customer 5 has arrived at 2.585652114727575
customer 5 has 2 before him
customer 3 ends up in the hospotal
customer 4 got to the ambulance
customer 4 ends up in the hospotal
customer 5 got to the ambulance
customer 6 has arrived at 3.3974061258901416
customer 6 has 1 before him
customer 5 ends up in the hospotal
customer 6 got to the ambulance
customer 6 ends up in the hospotal
customer 7 has arrived 

add multiplier, also after an event the whole simulation stops

In [9]:
def arrival(env,ambulance,trip,stop):
    i = 0
    while True:
        dt = expon.rvs(scale = 1/1)
        yield env.timeout(dt)
        print(f"customer {i} has arrived at {env.now}")
        env.process(service(env,ambulance,i,trip,stop))
        i += 1
        
def service(env,ambulance,i,trip,stop):
    rqt = ambulance.request()
    print(f"customer {i} has {len(ambulance.queue)} before him")
    yield rqt
    print(f"customer {i} got to the ambulance")
    t0 = env.now
    X = uniform.rvs()
    Y = uniform.rvs()
    multiplier = 2.25 * uniform.rvs() + 0.75
    dt = multiplier * np.sqrt((X-0.5)**2 + (Y-0.5)**2)
    yield env.timeout(dt)
    t1 = env.now
    print(f"customer {i} ends up in the hospotal")
    ambulance.release(rqt)
    trip.append(t1-t0)
    if  np.sum(trip) > 4:
        stop.succeed()
        yield stop

        


def system():
    env = simpy.Environment()
    trip = []
    ambulance = simpy.Resource(env,capacity = 1)    
    stop = simpy.Event(env)
    env.process(arrival(env,ambulance, trip,stop))
    env.run(until = stop)
    return trip

In [10]:
system()

customer 0 has arrived at 0.7341396191769328
customer 0 has 0 before him
customer 0 got to the ambulance
customer 0 ends up in the hospotal
customer 1 has arrived at 1.5057188650954394
customer 1 has 0 before him
customer 1 got to the ambulance
customer 2 has arrived at 2.3307409261544003
customer 2 has 1 before him
customer 3 has arrived at 2.792720271538062
customer 3 has 2 before him
customer 1 ends up in the hospotal
customer 2 got to the ambulance
customer 2 ends up in the hospotal
customer 3 got to the ambulance
customer 4 has arrived at 3.6353095030490885
customer 4 has 1 before him
customer 3 ends up in the hospotal
customer 4 got to the ambulance
customer 5 has arrived at 3.821543671819383
customer 5 has 1 before him
customer 4 ends up in the hospotal
customer 5 got to the ambulance
customer 5 ends up in the hospotal


[0.42996700242995634,
 1.5480005802817025,
 0.32391219492876955,
 0.419299636323494,
 0.510231297933391,
 1.1944964600355137]

When a call comes in, there is a timer, and if the timer runs out before ambulance heading towards that person, ambulance would drop them out by using rqt.cancel()

In [69]:
def arrival(env,ambulance,death):
    i = 0
    while True:
        dt = expon.rvs(scale = 1/1)
        yield env.timeout(dt)
        print(f"customer {i} has arrived at {env.now}")
        env.process(service(env,ambulance,i, death))
        i += 1
        
def service(env,ambulance,i, death):
    L = expon.rvs(scale = 4)
    rqt = ambulance.request()
    print(f"customer {i} has {len(ambulance.queue)} before him")
    result = yield rqt|env.timeout(L)
    if rqt not in result:
        rqt.cancel()
        death.append(i)
        print(f"customer {i} is dead")
    else:
        print(f"customer {i} got to the ambulance")
        X = uniform.rvs()
        Y = uniform.rvs()
        multiplier = 2.25 * uniform.rvs() + 0.75
        dt = multiplier * np.sqrt((X-0.5)**2 + (Y-0.5)**2) * 2
        yield env.timeout(dt)
        print(f"customer {i} ends up in the hospital")
        ambulance.release(rqt)


        


def system():
    env = simpy.Environment()
    death = []
    ambulance = simpy.Resource(env,capacity = 1)    
    env.process(arrival(env,ambulance, death))
    env.run(until = 24)
    return len(death)

In [74]:
system()

customer 0 has arrived at 0.5145653674298397
customer 0 has 0 before him
customer 0 got to the ambulance
customer 0 ends up in the hospital
customer 1 has arrived at 2.270182234220668
customer 1 has 0 before him
customer 1 got to the ambulance
customer 2 has arrived at 3.1911892729415836
customer 2 has 1 before him
customer 1 ends up in the hospital
customer 2 got to the ambulance
customer 3 has arrived at 4.57438582303759
customer 3 has 1 before him
customer 4 has arrived at 5.173698425798574
customer 4 has 2 before him
customer 2 ends up in the hospital
customer 3 got to the ambulance
customer 3 ends up in the hospital
customer 4 got to the ambulance
customer 5 has arrived at 7.466041358182437
customer 5 has 1 before him
customer 6 has arrived at 7.628253271209242
customer 6 has 2 before him
customer 4 ends up in the hospital
customer 5 got to the ambulance
customer 6 is dead
customer 7 has arrived at 8.871902292760087
customer 7 has 1 before him
customer 7 is dead
customer 8 has arr

8

If I don't get to pick up someone with Li, they die.
But even if they died afte I arrives, I will still heading to the hospital

In [82]:
def arrival(env,ambulance,death):
    i = 0
    while True:
        dt = expon.rvs(scale = 1/1)
        yield env.timeout(dt)
        print(f"customer {i} has arrived at {env.now}")
        env.process(service(env,ambulance,i, death))
        i += 1
        
def service(env,ambulance,i, death):
    L = expon.rvs(scale = 4)
    rqt = ambulance.request()
    t0 = env.now
    print(f"customer {i} has {len(ambulance.queue)} before him")
    result = yield rqt|env.timeout(L)
    if rqt not in result:
        rqt.cancel()
        death.append(i)
        print(f"customer {i} dies before ambulance")
    else:
        t1 = env.now
        passed = t1 - t0
        print(f"customer {i} got to the ambulance")
        X = uniform.rvs()
        Y = uniform.rvs()
        multiplier = 2.25 * uniform.rvs() + 0.75
        dt = multiplier * np.sqrt((X-0.5)**2 + (Y-0.5)**2) * 2
        die = env.timeout(L-passed)
        result = yield env.timeout(dt)| die
        if die in result:
            death.append(i)
            ambulance.release(rqt)
            print(f"customer {i} dies on the ambulance")
        else:
            print(f"customer {i} ends up in the hospital")
            ambulance.release(rqt)


        


def system():
    env = simpy.Environment()
    death = []
    ambulance = simpy.Resource(env,capacity = 1)    
    env.process(arrival(env,ambulance, death))
    env.run(until = 24)
    return len(death)

In [86]:
system()

customer 0 has arrived at 0.07712335653311571
customer 0 has 0 before him
customer 0 got to the ambulance
customer 1 has arrived at 0.7002646617556648
customer 1 has 1 before him
customer 2 has arrived at 1.3459956134472255
customer 2 has 2 before him
customer 0 dies on the ambulance
customer 1 got to the ambulance
customer 3 has arrived at 1.8278746778938868
customer 3 has 2 before him
customer 4 has arrived at 2.903176813051504
customer 4 has 3 before him
customer 1 ends up in the hospital
customer 2 got to the ambulance
customer 2 ends up in the hospital
customer 3 got to the ambulance
customer 4 dies before ambulance
customer 5 has arrived at 7.949506879727277
customer 5 has 1 before him
customer 3 ends up in the hospital
customer 5 got to the ambulance
customer 5 dies on the ambulance
customer 6 has arrived at 9.435345564007706
customer 6 has 0 before him
customer 6 got to the ambulance
customer 7 has arrived at 10.167279486289836
customer 7 has 1 before him
customer 8 has arrived

13

Let's figure out something. Ambulance cost 100/hr while it is travelling. Each patient pays 500. dies gives 0. Find the expected earning

In [97]:
def arrival(env,ambulance,death,trip,revenue):
    i = 0
    while True:
        dt = expon.rvs(scale = 1/1)
        yield env.timeout(dt)
        print(f"customer {i} has arrived at {env.now}")
        env.process(service(env,ambulance,i, death,trip,revenue))
        i += 1
        
def service(env,ambulance,i, death,trip,revenue):
    L = expon.rvs(scale = 4)
    rqt = ambulance.request()
    t0 = env.now
    print(f"customer {i} has {len(ambulance.queue)} before him")
    result = yield rqt|env.timeout(L)
    if rqt not in result:
        rqt.cancel()
        death.append(i)
        print(f"customer {i} dies before ambulance")
    else:
        t1 = env.now
        passed = t1 - t0
        print(f"customer {i} got to the ambulance")
        X = uniform.rvs()
        Y = uniform.rvs()
        multiplier = 2.25 * uniform.rvs() + 0.75
        dt = multiplier * np.sqrt((X-0.5)**2 + (Y-0.5)**2) * 2
        die = env.timeout(L-passed)
        trip.append(dt)
        result = yield env.timeout(dt)| die
        if die in result:
            death.append(i)
            ambulance.release(rqt)
            print(f"customer {i} dies on the ambulance")
        else:
            print(f"customer {i} ends up in the hospital")
            revenue.append(500)
            ambulance.release(rqt)


        


def system():
    env = simpy.Environment()
    death = []
    trip = []
    revenue = []
    ambulance = simpy.Resource(env,capacity = 1)    
    env.process(arrival(env,ambulance, death,trip,revenue))
    env.run(until = 24)
    return np.sum(revenue) + np.sum(trip) * -100

In [99]:
system()

customer 0 has arrived at 1.4594942302854892
customer 0 has 0 before him
customer 0 got to the ambulance
customer 1 has arrived at 1.714947709824084
customer 1 has 1 before him
customer 2 has arrived at 1.8224319409371146
customer 2 has 2 before him
customer 1 dies before ambulance
customer 3 has arrived at 2.452370936568779
customer 3 has 2 before him
customer 4 has arrived at 3.044819566838173
customer 4 has 3 before him
customer 0 ends up in the hospital
customer 2 got to the ambulance
customer 5 has arrived at 3.418781292756923
customer 5 has 3 before him
customer 6 has arrived at 3.6958597168163836
customer 6 has 4 before him
customer 2 dies on the ambulance
customer 3 got to the ambulance
customer 3 ends up in the hospital
customer 4 got to the ambulance
customer 6 dies before ambulance
customer 5 dies before ambulance
customer 4 ends up in the hospital
customer 7 has arrived at 7.268473130885663
customer 7 has 0 before him
customer 7 got to the ambulance
customer 8 has arrived a

1415.701394673103